In [311]:
#optional code if warnings become over bearing 
import warnings
warnings.filterwarnings('ignore')

In [312]:
%matplotlib inline

import os
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date
from datetime import datetime
from datetime import timedelta

from darksky import forecast

import config

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import TwoByTwoLayout
from ipywidgets import GridspecLayout

## Loading Models & Data

In [313]:
df_fires = pd.read_csv('data/FireIntensity_Model_June12_Clean.csv')

## Defining widgets

In [314]:
datepicker = widgets.DatePicker(
    description='Fire start: ',
    disabled=False,
    value = date.today()
)

In [315]:
temperature = widgets.IntSlider(value=85, max=120, min=20, description='Temperature: ')
temperature.style.handle_color = 'lightblue'

In [316]:
humidity = widgets.IntSlider(value=85, max=120, min=20, description='Humidity: ')

In [317]:
lat = widgets.FloatSlider(value=df_fires.latitude.median(),
                          max=df_fires.latitude.max(),
                          min=df_fires.latitude.min(),
                          description='Latitude: ',
                          readout_format='.4f',
                          step=0.0001,
                         )
lat.style.handle_color = 'lightpink'

lon = widgets.FloatSlider(value=df_fires.longitude.median(),
                          max=df_fires.longitude.max(),
                          min=df_fires.longitude.min(),
                          description='Longitude: ',
                          readout_format='.4f',
                          step=0.0001,
                         )
lon.style.handle_color = 'lightpink'

In [318]:
fuel_moist = widgets.Dropdown(
    options=[('Very Dry', 1), ('Dry', 2), ('Moderate', 3), ('Moist', 4)],
    value=1,
    description='Fuel Moisture Class:',
)

In [342]:
prefire = widgets.FloatSlider(value=df_fires.prefire_fuel.median(),
                          max=df_fires.prefire_fuel.max(),
                          min=df_fires.prefire_fuel.min(),
                          description='Prefire Fuel: ',
                          readout_format='.5f',
                          step=0.00001,
                         )
prefire.style.handle_color = 'lightblue'

In [336]:
fuel_code = widgets.Dropdown(
    options=[1600, 1220,    1, 1400, 1140, 1370, 1920,    2, 1160, 1500, 2700,
       2900, 1200, 1910, 2950, 2180, 1900, 1800, 1180, 1100, 1300, 1280,
       1260, 1120, 1950, 1940, 1360, 1320, 1340, 1700, 1980, 1990],
    value=1,
    description='Fuelcode:',
)

## Predicting Fire Intensity App

#### Enter the information in app below and submit for pridiction on fire intensity

In [387]:
fireapp = GridspecLayout(4, 3)
fireapp[0,0] = datepicker
#fireapp[0,2] = temperature
fireapp[1,0] = lat
fireapp[2,0] = lon
fireapp[0,1] = fuel_moist
fireapp[1,1] = fuel_code
fireapp[2,1] = prefire

In [388]:
fireapp

GridspecLayout(children=(DatePicker(value=datetime.date(2019, 11, 5), description='Fire start: ', layout=Layou…

In [320]:
#source: https://towardsdatascience.com/heres-how-to-calculate-distance-between-2-geolocations-in-python-93ecab5bbba4

# estimate fire region input
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

distances_km = []

for row in df_fires.itertuples(index=False):
   distances_km.append(
       haversine_distance(lat.value, lon.value, row.latitude, row.longitude)
   )

df_fires['DistanceFromPoint'] = distances_km

fire_region_in = df_fires[df_fires.DistanceFromPoint == df_fires.DistanceFromPoint.min()].fire_region.min()

In [321]:
#get doy input
def get_season(doy):
    if ((doy >= 80) and (doy <= 172)):
        s = 0  # spring
    elif ((doy > 172) and (doy < 264)):
        s = 1  # summer
    elif ((doy >= 264) and (doy <= 355)):
        s = 2  # fall
    #elif ((doy > 355) and (doy < 80)):
        #s = 3  # winter
    else:
        s = 3
     #   raise IndexError("Invalid date")
    return s

season_in = get_season(datepicker.value.timetuple().tm_yday)

In [370]:
def weather_lookup(date, lat, lon, key=config.darksky_api1):
    data = []    
    ts = date.isoformat() 
    lat = lat
    lon = lon
    weather = forecast(key, lat, lon, time=ts)
    w_dict = weather['currently']
    data.append(w_dict)
    return data

dt = datetime(datepicker.value.year, datepicker.value.month, datepicker.value.day, 12)

weather_df = pd.DataFrame(weather_lookup(dt, lat.value, lon.value, config.darksky_api1))

temperature_in = weather_df.temperature.min()
wind_speed_in = weather_df.windSpeed.min()
humidity_in = weather_df.humidity.min()
visibility_in = weather_df.visibility.min()
precip_intensity_in = weather_df.precipIntensity.min()
wind_gust_in = weather_df.windGust.min()

In [391]:
print("_____User Inputs_____")
print("Fire Start: ", datepicker.value)
print("Latitude: ", lat.value)
print("Longitude: ", lon.value)
print("Fuel Moisture Class: ", fuel_moist.value)
print("Prefire Fuel: ", prefire.value)
print("Fuelcode: ", fuel_code.value)
print("")


print("___Model Input Values___")
print("Temperature: ", temperature_in)
print("Humidity: ", humidity_in)
print("Visibility: ", visibility_in)
print("Precip Intensity: ", precip_intensity_in)
print("Wind Gust: ", wind_gust_in)
print("Wind Speed: ", wind_speed_in)
print("Season: ", season_in)
print("Fire Region: ", fire_region_in)
print("Fuel Moisture Class: ", fuel_moist.value)
print("Prefire Fuel: ", prefire.value)
print("Fuelcode: ", fuel_code.value)

_____User Inputs_____
Fire Start:  2019-11-05
Latitude:  30.4746
Longitude:  -86.2953
Fuel Moisture Class:  3
Prefire Fuel:  4523.28571
Fuelcode:  1260

___Model Input Values___
Temperature:  78.25
Humidity:  0.77
Visibility:  10
Precip Intensity:  0.0098
Wind Gust:  8.04
Wind Speed:  5.45
Season:  2
Fire Region:  2
Fuel Moisture Class:  3
Prefire Fuel:  4523.28571
Fuelcode:  1260


In [381]:
# pull from darksky api
'temperature', 'wind_speed','humidity','visibility', 'precip_intensity', 'wind_gust'

#use functions to create
'fire_region', 'season',

#set by user
'fuel_moisture_class','fuelcode', 'prefire_fuel'



('fuel_moisture_class', 'fuelcode', 'prefire_fuel')

In [307]:
## Other cool stuff to try

In [308]:

a = widgets.IntSlider(description='a')
a.style.handle_color = 'lightblue'
b = widgets.IntSlider(description='b')
b.style.handle_color = 'green'
c = widgets.IntSlider(description='c')
c.style.handle_color = 'red'
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])


In [309]:

widgets.ToggleButtons(
    options=['Slow', 'Regular', 'Fast'],
    description='Speed:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

ToggleButtons(description='Speed:', options=('Slow', 'Regular', 'Fast'), tooltips=('Description of slow', 'Des…

In [310]:

accordion = widgets.Accordion(children=[widgets.IntSlider(), widgets.Text("test")], titles=('Slider', 'Text'))
accordion

Accordion(children=(IntSlider(value=0), Text(value='test')))